Project Documenation

This assignment proved to be more complex than originally anticipated. Below are the decision-making processes that outline my approach. 


EDA (exploratory data analysis): 


Starting off with BigQuery and obtaining the proper fields necessary to approach this assignment, one of the first issues that the team came across was learning how to properly unnest the selected fields as many of the necessary fields were nested as recrods within the dataset. Nested fields do not work well within the pandas dataframe and would often lead to kernels crashing in our un-packing process. It is safe to assume that the team lost four days total just trying to figure this part out. My approach in the selected field names took a route of assessing 'hits' via a Session ID. That is to say, I wanted to assess whether any of the hits recorded around a particular pinged-session would lead to an event type of add to cart. As per the assignment rules, I took one month's worth of data, I specifically selected an odd-month between Nov. 20, 2016 and Dec 20, 2016. In my mind, this would have been the busiest time of the years for online shopping as both CyberMonday and Christmas shopping would have seen an uptick in sales during this time of year. Once I finally unnested the proper parameters I was able to move forward to the next part of the assignment. 

Without the use of matplotlib, the EDA of portion of this assignmet also proved to be troublesome. In this program training, matplotlib had always been an allowed dependency, and for this final assessment it was not. This meant that all my exploratory data analysis had to be learned on the spot, and I only managed to make a few countplots and histograms actually work. Unfortunately, I did not gather much correlation based on these and found it pragmatic to just keep moving forward with assignment. 


Data pre-processing:

The data pre-processing of this assignment, was to my surprise, much easier to organize. Much of the field_name unnesting took place on the BigQuery data-pooling. And outline of the Query is displayed below. 

SELECT 
concat(fullVisitorId, visitId) AS session_id,
visitNumber, CAST(SUBSTRING(`date`,5,2) AS int) AS month,
CAST(SUBSTRING(`date`,7,2) AS int) AS day_of_month,
totals.hits,
hit.hitNumber,
CASE WHEN totals.visits IS NULL THEN 0 ELSE 1 END AS visits,
CASE WHEN totals.bounces IS NULL THEN 0 ELSE 1 END AS bounces, 
CASE WHEN trafficSource.isTrueDirect IS NULL THEN 0 WHEN trafficSource.isTrueDirect IS FALSE THEN 0 ELSE 1 END AS isTrueDirect,
trafficSource.medium,
trafficSource.source,
CASE socialEngagementType WHEN 'Not Socially Engaged' THEN 0 ELSE 1 END AS social_Engagement,
channelGrouping,
device.deviceCategory,
promo.promoName,
CASE WHEN hit.promotionActionInfo.promoIsView IS NULL THEN 0 ELSE 1 END AS promoIsView,
CASE hit.eCommerceAction.action_type WHEN '3' THEN 1 ELSE 0 END AS add_to_cart
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  LEFT JOIN UNNEST (hits) AS hit
  LEFT JOIN UNNEST (hit.promotion) AS promo
WHERE 
  _TABLE_SUFFIX BETWEEN '20161120' AND '20161220'
ORDER BY session_id, hit.hitNumber

Via this method, the only parameter that was troublesome to unpack was the "promoName" field. My experience with SQL is limited, and I did not possess the understanding needed to unpack this field properly. Instead, I decided to carry the promoName field over to the Vertex AI work environment and employ vertex AI to unpack it properly. Once I unpacked the promoName field, I continued the data-preprocessing, which at this point had been properly unpacked on the BigQuery side. It was ideal too that I had no missing values in my dataset. The major pre-processing steps would include one-hot encoding the categorical columns so that they would work well with the various machine learning models. d


Feature Selection and Engineering:

At one point, post-OneHotEncoding, I ended up with 250 columns in my dataset and roughly 1.2 million rows. Naturally, I did not want to overfit my model, so the features below, and after various iterations of different features selected, amounted to the final selection that would go on to fit and train the models. 

    #From the Original Data Featues
    
    'hits', 'hitNumber', 'isTrueDirect', 'social_Engagement', 
    'promoIsView', 'promoOffice','promoLeather','promoElectro','promoMtshirts', 'promoFruit', 
    'promoAndroid', 'promoDrinks', 'promoOfficeCampaign', 'promoPenPencilHigh', 'promoLifestyle', 
    'promoAccessories', 'promoEngravedCeramic', 'promoBack', 'promoWtshirts', 'promoGoogle', 'promoYoutube', 
    'promoApparel', 'promoApparelCampaign',
    
    #Adding the One-Hot Encoding Features
    
    'medium_(none)', 'medium_affiliate', 'medium_cpm', 'medium_organic', 'medium_referral', 'source_(direct)', 'channelGrouping_Affiliates', 
    'channelGrouping_Direct', 'channelGrouping_Display', 'channelGrouping_Organic Search', 'channelGrouping_Paid Search', 'channelGrouping_Referral', 
    'channelGrouping_Social', 'deviceCategory_desktop', 'deviceCategory_mobile', 'deviceCategory_tablet'



Model Choice: 

Though I did not get around to deploying a model, I can at least speak to the various model iterations that I ran through, and at least, the final ones that ended up in writing on my capstone. My reason for choosing Linear Regression is that is highly interpretable, making it easy to understand the impact of each feauture on the predicted outcome of the target variable. This model is computationally efficient, the model training portion itself took just a few minutes and made it ideal to go back, make selected feature changes, and re-train the model to assess how the new parameters affected performance. Also, knowing that I could have employed either a small selected amount of data, or a large dataset, the scalability of Linear Regression models made it advantageuous when working against the clock. 

Overall, Logistic Regression would have made it to my deployment stage had it not been for my own faults in the approach and backtracking through my initial mistakes. Logistic Regression proved both versatile and easily interpretable  model that appeared to be efficient at predicting customer behaviors, in this case, the "add-to-cart" event. 


Model Evaluation Metrics

Despite my high hopes for this model, in hindssight,  I believe my initial approach was filled with bias. I'll explain, I specifically chose column field names that were associated with user-experience (sessionID), then I coupled that with more bias in the form of promotional adverstisements, such as social networking/engagement, referreral mediumns, and various promotions. 

I see now that this approach was a bit to simplistic. Without the need for machine learning models, one could easily deduce that if there is a certain promotion for a product, then it is more likely that someone will buy said product (i.e. "Add it to their Cart"). This model ended up being too simplistic, highly biased, and as a result I likely underfitted the training data. All in all, this led to poor predictive performance and lead to incredidibly high error rates. 

Another way of framing underfitting and high bias issue, is that the model then has limited flexibility to capture complex relationships within the data. This can be attributed to the data ONLY modeling after user experience based off of social engagement and promotional advertisements. It fails to understand complexities outside of that particular window, especially when working with a linear model. New and unseen data, in other words real-world data, would have performed just as poorly since the model was overly influence by the training data which was saturated with my own personal bias when selecting the columns fields in BigQuery. 



Model Tuning

For the linear regression model, I employed 'C', the inverse regulaization of strength) and 'penalty' (type of regularization), both of these were defined under the parameter grid "param_grid". I then coupled these hyperparameters under the Grid Search Cross-Validation. This cross validation is specifically useful in exploring the defined hyperparameter space that was previously defined  and will define the best comnination of hyperparameters that will yield the best performance. 

Once these hyperparameters were defined, they are then fitten back to the training model where the tuned model can then be evaluated. In general this should improve predictive performance and generalization for the model. However, I reiterate, that my field_name selection was highly flawed and led to underfitting. Despite the hyperparameter tuning, the penalty "l2", aka Ridge, is actually meant to prevent overfitting, but if my model was never overfitting, it may explain why after hyperparameter tuning, the overall evaluation and statistics did not change. 




o Depoloyment Performance Monitoring & Maintenance Plan -- Was not able to deploy a model in time, lack of proper understanding and poor model performance. 



Cost Management:

Unfortunately, without deploying a model and selecting all the appropriate features, I am limited in what I can write regarding the cost management of deploying and maintaining a model. Instead I wil speak generally of the need for budget analysis. 

Cost Management in deploying a model, involves meticulous tracking and an analysis of all the various resources utilized throughout the project lifecycles. This could also encompass not only deploymentment and maintenance of the model itself, but also all of the ancillary resources utilized to reach the end-point. That is to say, the creation of the Vertex AI instance, the BigQuery data-mining, and the various iterations undertaken to obtain the proper dataset necessary to train and test a model. 

Moreover, the reason for closely monitoring the expenses, from a business standpoint, is that decision-makers and stakeholders to these machine learning models are then able to leverage cost-effective virtual machine instances for non-critical tasks. As a result, these same stakeholders, can better distribute workload demands within their human-capital. 

Even though business can easily keep track of the expenses, especially when Google can bill users directly, what's more important in these budget tracking, is pin-pointing the investments made in resource optimaization strategies when working with digital resources. If it can be ascertained that a certain approach yields a higher output, with increased operational efficieny, all the meanwhile keeping costs low, then it would be both pragmatic and prudent for business leaders and stakeholders to make informed decisions in order to ensure proper resource allocation and macim